In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import keras.layers as klr
from keras.models import Model
import keras.applications as kapp
import keras.preprocessing.image as kimg
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
model = kapp.Xception(input_shape=(240, 240, 3), include_top=False, weights='imagenet')

for layer in model.layers:
    layer.trainable = False

x = model.output
x = klr.GlobalAveragePooling2D()(x)
x = klr.Dense(2, activation='softmax')(x)
model = Model(model.input, x)
model.summary()













83689472/83683744 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 240, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 119, 119, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 119, 119, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 119, 119, 32) 0           block1_conv1_bn[0][0]            
_______________

In [4]:
train_data = kimg.ImageDataGenerator(rescale=1/255).flow_from_directory('/content/drive/My Drive/DatasetForHackabit2019/Hollywood-dev/movies/pulp_fiction',
                                                                        target_size=(240, 240))

Found 0 images belonging to 0 classes.


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy')